# Definição de classes

In [5]:
# Estruturas criptográficas 2024-2025
# Grupo 02 - Miguel Ângelo Martins Guimarães (pg55986) e Pedro Miguel Oliveira Carvalho (pg55997)

# Imports
from sage.all import *

# Edwards class
class Ed(object):
    def __init__(self,p, a, d , ed = None):
        assert a != d and is_prime(p) and p > 3
        K        = GF(p) 
  
        A =  2*(a + d)/(a - d)
        B =  4/(a - d)
    
        alfa = A/(3*B) ; s = B

        a4 =  s^(-2) - 3*alfa^2
        a6 =  -alfa^3 - a4*alfa
        
        self.K = K
        self.constants = {'a': a , 'd': d , 'A':A , 'B':B , 'alfa':alfa , 's':s , 'a4':a4 , 'a6':a6 }
        self.EC = EllipticCurve(K,[a4,a6]) 
        
        if ed != None:
            self.L = ed['L']
            self.P = self.ed2ec(ed['Px'],ed['Py'])  # gerador do gru
        else:
            self.gen()
    
    def order(self):
        # A ordem prima "n" do maior subgrupo da curva, e o respetivo cofator "h" 
        oo = self.EC.order()
        n,_ = list(factor(oo))[-1]
        return (n,oo//n)
    
    def gen(self):
        L, h = self.order()       
        P = O = self.EC(0)
        while L*P == O:
            P = self.EC.random_element()
        self.P = h*P ; self.L = L
  
    def is_edwards(self, x, y):
        a = self.constants['a'] ; d = self.constants['d']
        x2 = x^2 ; y2 = y^2
        return a*x2 + y2 == 1 + d*x2*y2

    def ed2ec(self,x,y):      ## mapeia Ed --> EC
        if (x,y) == (0,1):
            return self.EC(0)
        z = (1+y)/(1-y) ; w = z/x
        alfa = self.constants['alfa']; s = self.constants['s']
        return self.EC(z/s + alfa , w/s)
    
    def ec2ed(self,P):        ## mapeia EC --> Ed
        if P == self.EC(0):
            return (0,1)
        x,y = P.xy()
        alfa = self.constants['alfa']; s = self.constants['s']
        u = s*(x - alfa) ; v = s*y
        return (u/v , (u-1)/(u+1))


# Points class of ED
class ed(object):
    def __init__(self,pt=None,curve=None,x=None,y=None):
        if pt != None:
            self.curve = pt.curve
            self.x = pt.x ; self.y = pt.y ; self.w = pt.w
        else:
            assert isinstance(curve,Ed) and curve.is_edwards(x,y)
            self.curve = curve
            self.x = x ; self.y = y ; self.w = x*y
    
    def eq(self,other):
        return self.x == other.x and self.y == other.y
    
    def copy(self):
        return ed(curve=self.curve, x=self.x, y=self.y)
    
    def zero(self):
        return ed(curve=self.curve,x=0,y=1)
    
    def sim(self):
        return ed(curve=self.curve, x= -self.x, y= self.y)
    
    def soma(self, other):
        a = self.curve.constants['a']; d = self.curve.constants['d']
        delta = d*self.w*other.w
        self.x, self.y  = (self.x*other.y + self.y*other.x)/(1+delta), (self.y*other.y - a*self.x*other.x)/(1-delta)
        self.w = self.x*self.y
        
    def duplica(self):
        a = self.curve.constants['a']; d = self.curve.constants['d']
        delta = d*(self.w)^2
        self.x, self.y = (2*self.w)/(1+delta) , (self.y^2 - a*self.x^2)/(1 - delta)
        self.w = self.x*self.y
        
    def mult(self, n):
        m = Mod(n,self.curve.L).lift().digits(2)   ## obter a representação binária do argumento "n"
        Q = self.copy() ; A = self.zero()
        for b in m:
            if b == 1:
                A.soma(Q)
            Q.duplica()
        return A
    
# x, y: coordenadas do ponto em GF(p)
# Retorna 32 bytes (little-endian) com o bit MSB do último byte indicando o sinal de x.
def encode_ed25519(x, y, p):
    x_int = int(x)
    y_int = int(y)
    sign_bit = x_int & 1  # 0 ou 1, dependendo da paridade de x

    # Converte y_int em 32 bytes little-endian
    y_bytes = y_int.to_bytes(32, byteorder='little')

    # Se sign_bit == 1, define o MSB do último byte
    if sign_bit == 1:
        y_bytes = bytearray(y_bytes)
        y_bytes[31] |= 0x80
        y_bytes = bytes(y_bytes)

    return y_bytes

# Implementação

In [ ]:
# Estruturas criptográficas 2024-2025
# Grupo 02 - Miguel Ângelo Martins Guimarães (pg55986) e Pedro Miguel Oliveira Carvalho (pg55997)

# Imports
from sage.all import *
import secrets
from cryptography.hazmat.primitives import hashes

# Edwards 22519
p = 2^255-19
K = GF(p)
a = K(-1)
d = -K(121665)/K(121666)

ed25519 = {
'b'  : 256,
'Px' : K(15112221349535400772501151409588531511454012693041857206046113283949847762202),
'Py' : K(46316835694926478169428394003475163141307993866256225615783033603165251855960),
'L'  : ZZ(2^252 + 27742317777372353535851937790883648493), ## ordem do subgrupo primo
'n'  : 254,
'h'  : 8
}
############ ver os domain parameteres


# Criação do par de chaves (Parametros)
b = 256 # Tamanho Especifico ao ed25519
requested_security_strenght = 128 # Tamanho Especifico ao ed25519
keysDigest = hashes.Hash(hashes.SHA512()) # Utilização do sha512 como hash (Especificação do ed25519)

########## <-------------------------------------------------------------------- IMPLEMENTAR DRBG MAYBE?
private_key = secrets.token_bytes(requested_security_strenght) # Passo 1 (Gerar chave privada) 

# Passo 2 (Calcular a hash da chave privada) 
keysDigest.update(private_key)
d_privateKey_hash = keysDigest.finalize()

# Passo 3 (Gerar a chave publica)
metade = d_privateKey_hash[:len(d_privateKey_hash)//2] # Obter a primeira metade do hash gerado

# Passo 3.1 ("The first 32 octets of H are interpreted as a little-endian integer...")
metade_byteArray = bytearray(metade)
metade_byteArray[0] &= 248       # Limpar bits 0,1,2 ("and 11111000" que faz com que os 3 primeiros bits se tornem 0) (O algoritmo utiliza notação little endian)
metade_byteArray[31] &= 127      # Limpar bit 7 ("and 01111111" que faz com que o ultimo bit se torne 0) (O algoritmo utiliza notação little endian)
metade_byteArray[31] |= 64       # Definir bit 6 a 0 ("or 01000000" que faz com que o penuultimo bit se torne 1 e mantem os restantes) (O algoritmo utiliza notação little endian)
metade_novo = bytes(metade_byteArray)

# Passo 4 ("Determine an integer s from hdigest1 using little-endian convention")
s = int.from_bytes(metade_novo, byteorder="little")

# Passo 5 ("Compute the point [s]G. The corresponding EdDSA public key Q is the encoding of the point [s]G")
# Criar objeto da curva
E = Ed(p, a, d, ed25519) # Criar instancia do edwards
gx, gy = E.ec2ed(E.P) # Obter coordenadas na notação edwards (x,y) 
G = ed(curve=E, x=gx, y=gy) # Representa o gerador
pub_point = G.mult(s) # Obter [s]G
public_key = encode_ed25519(pub_point.x, pub_point.y, p) ######## verificar este passo
print("Public key (hex):", public_key.hex())

Public key (hex): af301412664d33e0630ecedb83fd6f1f4d8b4f7d49db5175c422d83bf3bb4d5b


Exiting Sage (CPU time 0m0.86s, Wall time 84m21.95s).
